In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import wbdata
import matplotlib.pyplot as plt
import scipy.stats as st
from sympy import *
import time as dt
from sklearn.linear_model import LinearRegression
import warnings
from IPython.display import Math,Latex
from pandas.errors import SettingWithCopyWarning
from statsmodels.api import *
from statsmodels.stats.diagnostic import het_goldfeldquandt
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures,StandardScaler

def shapiro_wilk(df):
    # Apply Shapiro-Wilk test to each column
    for column in df.columns:
        stat, p_value = st.shapiro(df[column])
        print(f"Column: {column}")
        print(f"Shapiro-Wilk Test Statistic: {stat}, p-value: {p_value}")
        if p_value > 0.05:
            print("Data is likely normally distributed.")
        else:
            print("Data is likely not normally distributed.")
        print("\n")

def spearman(df,x,y):
    """_summary_

    Args:
        df (pd.Dataframe): data
        x (str): name of X column
        y (str): name of Y column

    Returns:
        pd.Dataframe: spearman matrix
    """
    return df[[x,y]].corr(method='spearman')

def kendall(df,x,y):
    """_summary_

    Args:
        df (pd.Dataframe): data
        x (str): name of X column
        y (str): name of Y column

    Returns:
        pd.Dataframe: kendall matrix
    """
    return df[[x,y]].corr(method='kendall')

def goldfeldquandt(df,x,y):
    """_summary_

    Args:
        df (pd.Dataframe): data
        x (str): name of X column
        y (str): name of Y column
    """
    gq_test=het_goldfeldquandt(df[[y]],df[[x]])
    print(f"F-statistic: {gq_test[0]}, p-value: {gq_test[1]}")
    
    return (gq_test[0],gq_test[1])

init_printing(use_unicode=True,use_latex=True)
def sm_model_outputs(model,untrust):
    r2=model.rsquared
    params=model.params
    nobs=model.nobs
    df=model.df_model
    tvalues=model.tvalues
    pvalues=model.pvalues
    f_pvalue=model.f_pvalue
    fvalue=model.fvalue
    untrust=0.05
    t_crit = st.t.ppf(1 - untrust/2, nobs - (df + 1))
    print(f'Коэффициент Детерминаци равен: {str(round(r2*100,3)).replace(".",",")}% .')
    print(f'Коэфициенты b при каждом члене равны соответственно: {("; ".join(list((params).transform(str)))).replace(".",",")} .')
    for i in range(len(params)):
        if pvalues.iloc[i]<untrust:
            print(f'Коэффициент {str(params.iloc[i]).replace(".",",")} стат. значим, т.к. значение t = {str(round(abs(tvalues.iloc[i]),4)).replace(".",",")} больше t_критического = {str(round(t_crit,4)).replace(".",",")} <=> pvalue={str(pvalues.iloc[i]).replace(".",",")} < {str(untrust).replace(".",",")}')
        else:
            print(f'Коэффициент {str(params.iloc[i]).replace(".",",")} стат. незначим, т.к. значение t = {str(round(abs(tvalues.iloc[i]),4)).replace(".",",")} меньше t_критического = {str(round(t_crit,4)).replace(".",",")} <=> pvalue={str(pvalues.iloc[i]).replace(".",",")} > {str(untrust).replace(".",",")}')
    print (f'Регрессия стат. значима, т.к. F-значение критерия фишера = {str(round(fvalue,4)).replace(".",",")} больше F_критического <=> fvalue={str(f_pvalue).replace(".",",")} < {str(untrust).replace(".",",")}')
 
df=pd.read_excel('Книга1 (3).xlsx',usecols=[9,10])
df
X=df[['X']]
y=df[['Y']]
df[['1']]=[1]*X.shape[1]
df